# Introdução

In [1]:
import os
import re
import time
import json
import folium
import random
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins
from osgeo import gdal, osr
from bs4 import BeautifulSoup
from tqdm.notebook import trange, tqdm

<br>

# Join Data

In [2]:
# Lê o arquivo csv com o nome dos municípios
df = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp_cetesb/main/data/tabs/tab_municipio_cetesb.csv',
)

# Lê o arquivo csv com o nome dos municípios
gdf = gpd.read_file(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/shps/sp_250k_wgs84.geojson',
)
gdf.drop(['municipio_nome'], axis=1, inplace=True)
gdf['id_municipio'] = gdf['id_municipio'].astype(int)
gdf['geometry'] = gdf.simplify(0.0015)

# Merge
gdf = gdf.merge(
    df,
    on='id_municipio',
    how='left'
)

# Results
gdf.head()

,id_municipio,geometry,municipio_nome,agencia,endereco,complemento,numero,bairro,municipio_sede,cep,telefone,fax,email,url
0,3500105,"POLYGON ((-51.05425 -21.40465, -51.04624 -21.4...",Adamantina,Agência Ambiental de Dracena,Rua Tomé de Souza,NaN,438,Centro,Dracena,17900-000,(18) 3822-3166,(18) 3822-3166,dracena_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
1,3500204,"POLYGON ((-49.65795 -21.20333, -49.65361 -21.2...",Adolfo,Agência Ambiental de São José do Rio Preto,Avenida Floriano André Cabrera,NaN,NaN,Residencial Cidade Jardim - Jardim São Marcos,São José do Rio Preto,15081-490,(17) 3218-4300,(17) 3218-4300,riopreto_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
2,3500303,"POLYGON ((-46.97640 -21.96818, -46.97399 -21.9...",Aguaí,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,NaN,345,Centro,São João da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
3,3500402,"POLYGON ((-46.73501 -21.81891, -46.72104 -21.8...",Águas da Prata,Agência Ambiental de São João da Boa Vista,Rua Marechal Deodoro,NaN,345,Centro,São João da Boa Vista,13870-223,(19) 3633-1510,(19) 3633-1510,sjboavista_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...
4,3500501,"POLYGON ((-46.60614 -22.44173, -46.59592 -22.4...",Águas de Lindóia,Agência Ambiental de Mogi Guaçu,Rua Hugo Pancieira,NaN,400,Pedregulhal,Mogi Guaçu,13845-190,(19) 3818-0396,(19) 3818-0396,mogiguacu_cetesb@sp.gov.br,https://licenciamento.cetesb.sp.gov.br/municip...


<br>

## Create pop up's column

In [3]:
# Add Field
def popup_html(row):
    tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    html = """
    <div>
    <p><b>{}</b> pertence à:
    <h4><b>{}</b></h4></p>
    
    <p><b>Sede:</b><br>
    {}{}{}<br>
    {}
    {}
    {}</p>
    <p><b>Contatos:</b><br>
    {}
    {}
    {}</p>
    <p>{}</p>
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome']) else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['agencia'])        else '{}'.format(row['agencia']),
        '' if pd.isnull(row['endereco'])       else '{}'.format(row['endereco']),
        '' if pd.isnull(row['numero'])         else ', {}'.format(row['numero']),
        '' if pd.isnull(row['complemento'])    else ' - {}'.format(row['complemento']),
        '' if pd.isnull(row['bairro'])         else 'Bairro: {}<br>'.format(row['bairro']),
        '' if pd.isnull(row['municipio_sede']) else 'Município: {}<br>'.format(row['municipio_sede']),
        '' if pd.isnull(row['cep'])            else 'CEP: {}<br>'.format(row['cep']),
        '' if pd.isnull(row['telefone'])       else 'Telefone: <a href="tel:{}">{}</a><br>'.format(tel, row['telefone']),
        '' if pd.isnull(row['fax'])            else 'Fax: <a href="tel:{}">{}</a><br>'.format(fax, row['fax']),
        '' if pd.isnull(row['email'])          else 'E-mail: <a href="mailto:{}">{}</a>'.format(row['email'], row['email']),
        '' if pd.isnull(row['email'])          else '<a href="{}" target="_blank">Conferir Dados</a>'.format(row['url']),
    )
    
    html = html.replace('\n','')
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

In [4]:
# Calculate PopUps
gdf['popup'] = gdf.apply(popup_html, axis=1)

<br>

## Adjust Table

In [5]:
# Delete Columns
gdf.drop([
    'id_municipio', 'endereco', 'numero', 'bairro',
    'municipio_sede', 'cep', 'telefone', 'fax',
    'complemento', 'url', 'email',], axis=1, inplace=True)
print(gdf.columns)

Index(['geometry', 'municipio_nome', 'agencia', 'popup'], dtype='object')


In [6]:
# Save geojson
gdf.to_file(
    os.path.join('data', 'shps', 'sp_cetesb.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

# Results
gdf.head()

,geometry,municipio_nome,agencia,popup
0,"POLYGON ((-51.05425 -21.40465, -51.04624 -21.4...",Adamantina,Agência Ambiental de Dracena,<div> <p><b>Adamantina</b> pertence à: <h4><b>...
1,"POLYGON ((-49.65795 -21.20333, -49.65361 -21.2...",Adolfo,Agência Ambiental de São José do Rio Preto,<div> <p><b>Adolfo</b> pertence à: <h4><b>Agên...
2,"POLYGON ((-46.97640 -21.96818, -46.97399 -21.9...",Aguaí,Agência Ambiental de São João da Boa Vista,<div> <p><b>Aguaí</b> pertence à: <h4><b>Agênc...
3,"POLYGON ((-46.73501 -21.81891, -46.72104 -21.8...",Águas da Prata,Agência Ambiental de São João da Boa Vista,<div> <p><b>Águas da Prata</b> pertence à: <h4...
4,"POLYGON ((-46.60614 -22.44173, -46.59592 -22.4...",Águas de Lindóia,Agência Ambiental de Mogi Guaçu,<div> <p><b>Águas de Lindóia</b> pertence à: <...


## Get Centroid

In [7]:
def get_centroid(gdf):
    gdf['apenasparacentroid'] = 35
    gdf_dissolve = gdf.dissolve(by='apenasparacentroid')
    gdf_centroid = gdf_dissolve.representative_point()
    gdf = gdf.drop('apenasparacentroid', axis=1)
    return [float(gdf_centroid.y), float(gdf_centroid.x)]

In [8]:
list_centroid = get_centroid(gdf)
list_centroid

[-22.545968889465207, -49.56185387118866]

## Folium

In [24]:
def map_bomb(input_geojson, bbox):
    gdf = gpd.read_file(input_geojson)
    
    # Column with category
    col_categories = 'agencia'
    
    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    color_polygon['Agência Ambiental de Ribeirão Preto'] = '#e699ff'
    color_polygon['Agência Ambiental de Barretos'] = '#4da6ff'
    color_polygon['Agência Ambiental de São Carlos'] = '#ff5050'
    color_polygon['Agência Ambiental de Osasco'] = '#00264d'

    # Create Map
    m = folium.Map(
        location=[-22.545968889465207, -49.56185387118866],
        zoom_start=6,
        min_zoom=6,
        max_zoom=11,
        max_bounds=True,
        min_lon = bbox['min_lon']*(101/100),
        max_lon = bbox['max_lon']*(99/100),        
        min_lat = bbox['min_lat']*(101/100),
        max_lat = bbox['max_lat']*(99/100),
        #zoom_delta=0.1,
    )
    folium.GeoJson(
        gdf,
        name='CETESB',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        style_function=lambda x: {
            'fillColor': color_polygon[x['properties'][col_categories]],
            'color': color_polygon[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'agencia'],
            aliases=['Munícipio', 'Agência'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,            
        )        
    ).add_to(m)

    # Plugins
    m.fit_bounds(m.get_bounds())
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    return m

In [25]:
# Map without Bounds
bbox = {
    'max_lat': 0,
    'max_lon': 0,
    'min_lat': 0,
    'min_lon': 0,
}
m = map_bomb(os.path.join('data', 'shps', 'sp_cetesb.geojson'), bbox)

# Map with Bounds
bbox = {
    'max_lat': m.get_bounds()[1][0],
    'min_lat': m.get_bounds()[0][0],
    'max_lon': m.get_bounds()[1][1],
    'min_lon': m.get_bounds()[0][1],
}
m = map_bomb(os.path.join('data', 'shps', 'sp_cetesb.geojson'), bbox)

# Results
print(bbox)
m

{'max_lat': -19.779655795, 'min_lat': -25.357940277, 'max_lon': -44.161365164, 'min_lon': -53.10904723}


<br>

# Save Map

In [26]:
os.makedirs('maps', exist_ok=True)
m.save(os.path.join('maps', 'cetesb_map.html'))
m.save(os.path.join('..', '..', '..', 'case_django', 'divadmin', 'templates', 'cetesb_map.html'))